# Example of Probabilistic Graphical Model - Estimation of a rainy day

We want to estimate the probability of having a rainy day, but the room we are in, has no windows.

We can observe, however, the status of a particular person. We know that if it is rainy, there is a good chance that the person will bring an umbrella. Also we know that if it is raining and the person did not brough an umbrella, his/her clothes will be wet.

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [29]:
# Model definition
model = BayesianNetwork([
    ('Rain', 'Umbrella'),
    ('Rain', 'ClothesWet'),
    ('Umbrella', 'ClothesWet')
])

# Rain is our a priori variable. We decide to give rain a 50% chance of rain.
cpd_rain = TabularCPD(variable='Rain', variable_card=2, values=[[0.5], [0.5]], state_names={'Rain': ['yes', 'no']})

# The person brings the umbrella, when it is rain 90% of time. However, even if it does not rain, this person can bring an umbrella 70% of time
cpd_umbrella = TabularCPD(
    variable='Umbrella', variable_card=2,
    values=[[0.9, 0.5], [0.1, 0.5]],
    evidence=['Rain'], evidence_card=[2],
    state_names={'Umbrella': ['yes', 'no'], 'Rain': ['yes', 'no']}
)

# If it rainy and it has an umbrella, then we have 10% chance of still getting wet
cpd_clotheswet = TabularCPD(
    variable='ClothesWet', variable_card=2,
    values=[
        [0.05, 0.99, 0.001, 0.001],  # P(Wet=yes)
        [0.95, 0.01, 0.999, 0.999]   # P(Wet=no)
    ],
    evidence=['Rain', 'Umbrella'], evidence_card=[2, 2],
    state_names={'ClothesWet': ['yes', 'no'], 'Rain': ['yes', 'no'], 'Umbrella': ['yes', 'no']}
)

In [30]:
print(cpd_rain);
print(cpd_umbrella);
print(cpd_clotheswet);

+-----------+-----+
| Rain(yes) | 0.5 |
+-----------+-----+
| Rain(no)  | 0.5 |
+-----------+-----+
+---------------+-----------+----------+
| Rain          | Rain(yes) | Rain(no) |
+---------------+-----------+----------+
| Umbrella(yes) | 0.9       | 0.5      |
+---------------+-----------+----------+
| Umbrella(no)  | 0.1       | 0.5      |
+---------------+-----------+----------+
+-----------------+---------------+--------------+---------------+--------------+
| Rain            | Rain(yes)     | Rain(yes)    | Rain(no)      | Rain(no)     |
+-----------------+---------------+--------------+---------------+--------------+
| Umbrella        | Umbrella(yes) | Umbrella(no) | Umbrella(yes) | Umbrella(no) |
+-----------------+---------------+--------------+---------------+--------------+
| ClothesWet(yes) | 0.05          | 0.99         | 0.001         | 0.001        |
+-----------------+---------------+--------------+---------------+--------------+
| ClothesWet(no)  | 0.95          | 0.0

In [ ]:
model.add_cpds(cpd_rain, cpd_umbrella, cpd_clotheswet)
model.check_model()

# Inference
infer = VariableElimination(model)
result = infer.query(variables=['Rain'], evidence={'ClothesWet': 'yes'})
print(result)

In [31]:
# What if we were only looking at the status of the clothes
result = infer.query(variables=['Rain'], evidence={'ClothesWet': 'no'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.4481 |
+-----------+-------------+
| Rain(no)  |      0.5519 |
+-----------+-------------+


In [33]:
# What if we were only looking at the status of the clothes
result = infer.query(variables=['Rain'], evidence={'ClothesWet': 'yes'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.9947 |
+-----------+-------------+
| Rain(no)  |      0.0053 |
+-----------+-------------+


In [34]:
result = infer.query(variables=['Rain'], evidence={'ClothesWet': 'no', 'Umbrella': 'no'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.0033 |
+-----------+-------------+
| Rain(no)  |      0.9967 |
+-----------+-------------+


In [35]:
result = infer.query(variables=['Rain'], evidence={'ClothesWet': 'no', 'Umbrella': 'yes'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.5367 |
+-----------+-------------+
| Rain(no)  |      0.4633 |
+-----------+-------------+


In [36]:
result = infer.query(variables=['Rain'], evidence={'ClothesWet': 'yes', 'Umbrella': 'no'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.9970 |
+-----------+-------------+
| Rain(no)  |      0.0030 |
+-----------+-------------+


In [38]:
result = infer.query(variables=['Rain'], evidence={'ClothesWet': 'yes', 'Umbrella': 'yes'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.9923 |
+-----------+-------------+
| Rain(no)  |      0.0077 |
+-----------+-------------+


In [40]:
# what if we only look at the umbrella
result = infer.query(variables=['Rain'], evidence={'Umbrella': 'no'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.2500 |
+-----------+-------------+
| Rain(no)  |      0.7500 |
+-----------+-------------+


In [41]:
result = infer.query(variables=['Rain'], evidence={'Umbrella': 'yes'})
print(result)

+-----------+-------------+
| Rain      |   phi(Rain) |
+===========+=============+
| Rain(yes) |      0.5625 |
+-----------+-------------+
| Rain(no)  |      0.4375 |
+-----------+-------------+
